In [1]:
import pandas as pd
import importlib
import geopandas as gpd
from shapely.geometry.point import Point
import networkx as nx
import osmnx as ox

from cabby.data.wikidata import query
from cabby.geo.map_processing.map_processor import map_structure
import cabby.geo.util as util
from cabby.data.metagraph import utils

In [2]:
# Get the existing map
base_osm_map_filepath = '/mnt/hackney/data/cabby/map/v14'
s2_level = 18
region = "Pittsburgh"
osm_map = map_structure.Map(region, s2_level, base_osm_map_filepath)

In [3]:
# Prepare a new POI and prepare a dataframe row
osmid = 999111
new_df = pd.DataFrame(data={
  'name': 'Byers-Lyons House',
  'geometry': [util.point_str_to_shapely_point('Point(-80.016111 40.449722)')],
  'osmid': osmid,
  'wikidata': 'Q5004063'
}, index=[osmid])
new_df.index.rename('osmid', inplace=True)
single_poi = new_df.loc[osmid]

In [6]:
# Run first part of map_structure.Map.add_single_poi_to_graph,
# and get the point to feed to add_single_point_edge
geometry = single_poi['geometry']
if isinstance(geometry, Point):
    points = [single_poi['geometry']]
elif isinstance(geometry, Polygon):
    coords = single_poi['geometry'].exterior.coords
    n_points = len(coords)
    
    # Sample maximum 4 points.
    sample_1 = Point(coords[0])
    sample_2 = Point(coords[round(n_points/4)])
    sample_3 = Point(coords[round(n_points/2)])
    sample_4 = Point(coords[round(3*n_points/4)])
    points = [sample_1, sample_2, sample_3, sample_4]
    points = points[0:4]
else:
    raise UnknownError

poi_osmid = single_poi['osmid']
poi_osmid = util.concat_numbers(999, poi_osmid)
assert poi_osmid not in osm_map.poi['osmid'].tolist(), poi_osmid
osm_map.poi.loc[osm_map.poi['osmid'] ==
       single_poi['osmid'], 'osmid'] = poi_osmid

list_edges_connected_ids = []
edges_to_add = []
point = points[0]

In [7]:
# Show arguments to map_structure.Map.add_single_point_edge
print(point)
print(list_edges_connected_ids)
print(poi_osmid)

POINT (-80.016111 40.449722)
[]
999999111


In [10]:
# Run map_structure.Map.add_single_point_edge up to assertion error
try:

    near_edge_u, near_edge_v, near_edge_key, line = \
        ox.distance.get_nearest_edge(
            osm_map.nx_graph, util.tuple_from_point(point), return_geom=True, 
        )

except Exception as e:
    print(e)
    # return []

edge_id = (near_edge_u, near_edge_v, near_edge_key)

if edge_id in list_edges_connected_ids:  # Edge already connected
    # return []
    raise UnknownError

# Get nearest points - u and v.
u_node = osm_map.nx_graph.nodes[near_edge_u]
u_point = Point(u_node['x'], u_node['y'])

v_node = osm_map.nx_graph.nodes[near_edge_v]
v_point = Point(v_node['x'], v_node['y'])

# Add to connected edges.
list_edges_connected_ids.append(edge_id)

near_edge = osm_map.nx_graph.edges[edge_id]

dist_projected = line.project(point)
projected_point = line.interpolate(line.project(point))

cut_geometry = util.cut(line,dist_projected)

n_lines = len(cut_geometry) 

line_1 = cut_geometry[0]
dist_1 = util.get_line_length(line_1)

if n_lines==2:
    assert projected_point==Point(line_1.coords[-1])
    line_2 = cut_geometry[1]
    dist_2 = util.get_line_length(line_2)

    projected_point_osmid = util.concat_numbers(
    len(list_edges_connected_ids), poi_osmid)

else: # Projected point is exactly on the end of the line (U or V).
    dist_u_p = util.get_distance_between_points(u_point, projected_point)
    dist_v_p = util.get_distance_between_points(v_point, projected_point)
    assert dist_u_p==0 or dist_v_p==0
    if dist_u_p==0:
        projected_point_osmid = near_edge_u
    else:
        projected_point_osmid = near_edge_v

AssertionError: 

In [11]:
dist_u_p

18.28590788244519

In [12]:
dist_v_p

28.377229757843207